In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('tr_eikon_eod_data.csv', sep=',')
df.head()

FileNotFoundError: File b'tr_eikon_eod_data.csv' does not exist

In [ ]:
symbol = "AAPL.O"
data = df[["Date", symbol]].dropna()
data.head()

In [ ]:
short_term_window = 40
long_term_window = 190
data["Date"] =  pd.to_datetime(data["Date"], format="%Y-%m-%d")
data["SMA_CT"] = data[symbol].rolling(short_term_window).mean()
data["SMA_LT"] = data[symbol].rolling(long_term_window).mean()
data["Return"] = np.log(data[symbol]) - np.log(data[symbol].shift(1))
data["Position"] = np.sign(data["SMA_CT"] - data["SMA_LT"])
data["Strategy"] = data["Return"] * data["Position"]
# data = data.dropna()
# data["Cumsum_return"] = data["Return"].cumsum()
# data["Cumsum_strategy"] = data["Strategy"].cumsum()
data.head()

In [ ]:
# Building the LSTM model

# Let's load the required libs.
# We'll be using the Tensorflow backend (default).
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle



In [ ]:
data_model = data[["Return"]].dropna() # TODO: turn the data to dataframe
# data_1_period = data_model[:1000]
data_1_period = data_model[-2000:]
data_1_period.head()

In [ ]:
# Get the raw data values from the pandas data frame.
data_raw = data_1_period.values.astype("float32")
dataset = data_raw
# We apply the MinMax scaler from sklearn
# to normalize data in the (0, 1) interval.
# scaler = MinMaxScaler(feature_range = (0, 1))
# dataset = scaler.fit_transform(data_raw)

# Print a few values.
dataset[0:5]

In [ ]:
len(dataset)

In [ ]:
# Train test split
TRAIN_SIZE = 0.75

train_size = int(len(dataset) * TRAIN_SIZE)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
print("Number of entries (training set, test set): " + str((len(train), len(test))))

In [ ]:
train

In [ ]:
# FIXME: This helper function should be rewritten using numpy's shift function. See below.
def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))


In [ ]:
# Create test and training sets for one-step-ahead regression.
window_size = 20
train_X, train_Y = create_dataset(train, window_size)
test_X, test_Y = create_dataset(test, window_size)
print("Original training data shape:")
print(train_X.shape)

# Reshape the input data into appropriate form for Keras.
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("New training data shape:")
print(train_X.shape)


In [ ]:
def fit_model(train_X, train_Y, window_size = 1):
    model = Sequential()
    model.add(LSTM(32, input_shape = (1, window_size)))    
#     model.add(LSTM(32,return_sequences=True, input_shape = (1, window_size)))
#     model.add(Dropout(0.5))
#     model.add(LSTM(16))
#     model.add(Dropout(0.5))
#     model.add(LSTM(32))
#     model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "RMSProp")
    model.fit(train_X, 
              train_Y, 
              epochs = 5, 
              batch_size = 1, 
              verbose = 2)
    
    return(model)

# Fit the first model.
model1 = fit_model(train_X, train_Y, window_size)

In [ ]:
def predict_and_score(model, X, Y):
#     pred = scaler.inverse_transform(model.predict(X))
#     orig_data = scaler.inverse_transform([Y])
    pred = model.predict(X)
    orig_data = [Y]
#     print(len(model.predict(X)))
#     print(pred)
#     print(len(pred))
#     print(len([Y]))
#     print(orig_data)
#     print(len(orig_data))
    score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
    return(score, pred)

rmse_train, train_predict = predict_and_score(model1, train_X, train_Y)
rmse_test, test_predict = predict_and_score(model1, test_X, test_Y)

print("Training data score: %.2f RMSE" % rmse_train)
print("Test data score: %.2f RMSE" % rmse_test)

In [ ]:
# Start with training predictions.
train_predict_plot = np.empty_like(dataset)
train_predict_plot[:, :] = np.nan
train_predict_plot[window_size:len(train_predict) + window_size, :] = train_predict

# Add test predictions.
test_predict_plot = np.empty_like(dataset)
test_predict_plot[:, :] = np.nan
test_predict_plot[len(train_predict) + (window_size * 2) + 1:len(dataset) - 1, :] = test_predict

# Create the plot.
plt.figure(figsize = (15, 5))
plt.plot(dataset, label = "True value")
# plt.plot(scaler.inverse_transform(dataset), label = "True value")
plt.plot(train_predict_plot, label = "Training set prediction")
plt.plot(test_predict_plot, label = "Test set prediction")
plt.xlabel("Days")
plt.ylabel("Return")
plt.title("Comparison true vs. predicted training / test")
plt.legend()
plt.show()

In [ ]:
data_final = data[-len(test_predict):]
data_final["Predicted_return"] = test_predict
data_final["Position_LSTM"] = np.sign(data_final["Predicted_return"])
data_final["Strategy_LSTM"] = data_final["Return"] * data_final["Position_LSTM"]
data_final["Cumsum_return"] = data_final["Return"].cumsum()
data_final["Cumsum_return_naive"] = data_final["Strategy"].cumsum()
data_final["Cumsum_strategy_LSTM"] = data_final["Strategy_LSTM"].cumsum()
data_final.head(100)

In [ ]:
data_final.tail(100)

The real return is 0.56, the returns of LSTM is 0.52. We can see that this LSTM model do not outperform the market yet. However, this LSTM model is still naive and there's much room for improvement. For example:
1. Divide the dataset into periods and train a model for each periods
2. Use a better strategy to decide long/short position
3. Tune LSTM model
4. Try another formula of return